In [1]:
#!pip install tensorflow==2.0


In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
project_path='/content/drive/My Drive/Colab Notebooks/'

In [5]:
datasetfile = project_path + 'bank.csv'

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
bank_cust = pd.read_csv(datasetfile)

In [8]:
bank_cust.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
bank_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [10]:
bank_cust.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [11]:
import seaborn as sns
#sns.pairplot(data=bank_cust)

In [12]:
#bank_cust.groupby(bank_cust['Geography']).mean()
bank_cust['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [13]:
#Row Number, Surname and Customer ID are unique for all customers and will therefore be dropped
bank_cust.drop(['Surname','RowNumber','CustomerId'],axis=1,inplace=True)

In [14]:
#one Hot encoding of categorical variables
bank_cust = pd.get_dummies(bank_cust) 

In [15]:
bank_cust.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
CreditScore,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094,-0.008928,0.005538,0.004780,0.002857,-0.002857
Age,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323,-0.039208,0.046897,-0.001685,0.027544,-0.027544
Tenure,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001,-0.002848,-0.000567,0.003868,-0.014733,0.014733
Balance,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533,-0.231329,0.401110,-0.134892,-0.012087,0.012087
NumOfProducts,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820,0.001230,-0.010419,0.009039,0.021859,-0.021859
HasCrCard,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138,0.002467,0.010577,-0.013480,-0.005766,0.005766
IsActiveMember,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128,0.003317,-0.020486,0.016732,-0.022544,0.022544
EstimatedSalary,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097,-0.003332,0.010297,-0.006482,0.008112,-0.008112
Exited,-0.027094,0.285323,-0.014001,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000,-0.104955,0.173488,-0.052667,0.106512,-0.106512
Geography_France,-0.008928,-0.039208,-0.002848,-0.231329,0.001230,0.002467,0.003317,-0.003332,-0.104955,1.000000,-0.580359,-0.575418,-0.006772,0.006772


In [16]:
#Distinguish features and target variable
Feature_data = bank_cust.drop('Exited',axis=1)
Target_data = bank_cust.pop('Exited')

In [17]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix,precision_score,recall_score,auc,precision_recall_curve
X_train,X_test,Y_train,Y_test = train_test_split(Feature_data,Target_data,test_size=.3,random_state=10)

In [18]:
#Import essential tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras import optimizers

In [19]:
#Normalize the train and test data
min_max_scaler = preprocessing.MinMaxScaler()
#X_train = preprocessing.normalize(X_train)
X_train = min_max_scaler.fit_transform(X_train)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7000, 13)
(3000, 13)
(7000,)
(3000,)


In [21]:
model = Sequential()

In [22]:
#Hidden keras layers are added
#Sigmoid layer is also added
model.add(Dense(120,input_shape=(13,),activation='tanh'))
model.add(Dense(110,activation='tanh'))
model.add(Dense(100,activation='tanh'))
model.add(Dense(90,activation='tanh'))
model.add(Dense(80,activation='tanh'))
model.add(Dense(70,activation='tanh'))
model.add(Dense(60,activation='tanh'))
model.add(Dense(50,activation='tanh'))
model.add(Dense(40,activation='tanh'))
model.add(Dense(30,activation='tanh'))
model.add(Dense(20,activation='tanh'))
model.add(Dense(10,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))

In [23]:
sgd = optimizers.Nadam(lr = 0.001)

In [24]:
#model compilation with loss function
model.compile(optimizer = sgd, loss='binary_crossentropy',metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 120)               1680      
_________________________________________________________________
dense_1 (Dense)              (None, 110)               13310     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               11100     
_________________________________________________________________
dense_3 (Dense)              (None, 90)                9090      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                7280      
_________________________________________________________________
dense_5 (Dense)              (None, 70)                5670      
_________________________________________________________________
dense_6 (Dense)              (None, 60)                4

In [26]:
#Training model
model.fit(X_train,Y_train.values,batch_size=400,epochs=150,verbose=1)

Epoch 1/150
18/18 [==============================] - 0s 8ms/step - loss: 0.4719 - accuracy: 0.7967
Epoch 2/150
18/18 [==============================] - 0s 8ms/step - loss: 0.4498 - accuracy: 0.8054
Epoch 3/150
18/18 [==============================] - 0s 8ms/step - loss: 0.4278 - accuracy: 0.8186
Epoch 4/150
18/18 [==============================] - 0s 8ms/step - loss: 0.4212 - accuracy: 0.8217
Epoch 5/150
18/18 [==============================] - 0s 7ms/step - loss: 0.4098 - accuracy: 0.8289
Epoch 6/150
18/18 [==============================] - 0s 7ms/step - loss: 0.4067 - accuracy: 0.8279
Epoch 7/150
18/18 [==============================] - 0s 7ms/step - loss: 0.3959 - accuracy: 0.8297
Epoch 8/150
18/18 [==============================] - 0s 8ms/step - loss: 0.3839 - accuracy: 0.8421
Epoch 9/150
18/18 [==============================] - 0s 8ms/step - loss: 0.3736 - accuracy: 0.8489
Epoch 10/150
18/18 [==============================] - 0s 7ms/step - loss: 0.3665 - accuracy: 0.8447
Epoch 11/

In [27]:
X_test = min_max_scaler.transform(X_test)
#X_test = preprocessing.normalize(X_test)

In [28]:
results = model.evaluate(X_test,Y_test.values)

94/94 [==============================] - 0s 1ms/step - loss: 0.7606 - accuracy: 0.8030


In [29]:
print(model.metrics_names)
print(results)

['loss', 'accuracy']
[0.7606339454650879, 0.8029999732971191]


In [33]:
Ypred_class = model.predict_classes(X_test,batch_size=200,verbose=0) #at default 0.5 threshold for binary classification
print("Recall Score: "+str(recall_score(Y_test.values,Ypred_class)))
print("Precision Score: "+str(precision_score(Y_test.values,Ypred_class)))
print("F-Score: "+str(f1_score(Y_test.values,Ypred_class)))
confusion_matrix(Y_test.values,Ypred_class)

Recall Score: 0.5435483870967742
Precision Score: 0.5224806201550387
F-Score: 0.532806324110672


array([[2072,  308],
       [ 283,  337]])

**Points of Improvements**
*   Activation Function changed to tanh
*   Several Hidden layers were added to improve accuracy
*   Epoch was increased to ensure better results











```
# This is formatted as code
```


